In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['MOS']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [46]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 0]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [47]:
list(zip(longforms, counts))

[('mannan oligosaccharide', 54),
 ('mannanoligosaccharides', 19),
 ('mitochondrial oxidative stress', 15),
 ('mannooligosaccharides', 14),
 ('mosapride citrate', 13),
 ('margin of stability', 10),
 ('main olfactory system', 6),
 ('maltooligosaccharide', 6),
 ('medical outcome study', 5),
 ('manno oligosaccharide', 5),
 ('metal oxidative semiconductor', 4),
 ('motor optimality score', 4),
 ('medical outcome survey', 3),
 ('moderate intensity continuous swimming', 3),
 ('mos', 2),
 ('mosaicplasty', 2),
 ('margin of safety', 2),
 ('maxillary occlusal splint', 2),
 ('mannose rich oligosaccharide', 2),
 ('malto oligo saccharide', 1),
 ('mannan oligo saccharide', 1),
 ('median overall survival', 1),
 ('metastatic osteosarcoma', 1),
 ('metrical oxidative state', 1),
 ('microvascular occlusal syndromes', 1),
 ('minor oral surgery', 1),
 ('mandibular occlusal splint', 1),
 ('mantis operating system', 1),
 ('metmyoglobin oxidative system', 1),
 ('malto oligosaccharide', 1),
 ('mannose oligosacch

In [7]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [8]:
names

{'CHEBI:CHEBI:50699': 'oligosaccharide',
 'MESH:D018384': 'Oxidative Stress',
 'CHEBI:CHEBI:25174': 'mannooligosaccharide',
 'CHEBI:CHEBI:31867': 'Mosapride citrate',
 'CHEBI:CHEBI:17593': 'maltooligosaccharide',
 'MESH:D012666': 'Semiconductors',
 'GO:GO:0036268': 'swimming',
 'HGNC:7199': 'MOS',
 'HGNC:18234': 'MOCOS'}

In [9]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-04 02:42:08] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [10]:
result = [grounding_map, names, pos_labels]

In [11]:
result

[{'main olfactory system': 'ungrounded',
  'maltooligosaccharide': 'CHEBI:CHEBI:17593',
  'mannan oligosaccharide': 'CHEBI:CHEBI:25174',
  'mannanoligosaccharides': 'CHEBI:CHEBI:25174',
  'manno oligosaccharide': 'CHEBI:CHEBI:25174',
  'mannooligosaccharides': 'CHEBI:CHEBI:25174',
  'margin of stability': 'ungrounded',
  'medical outcome study': 'ungrounded',
  'medical outcome survey': 'ungrounded',
  'metal oxidative semiconductor': 'ungrounded',
  'mitochondrial oxidative stress': 'MESH:D018384',
  'moderate intensity continuous swimming': 'GO:GO:0036268',
  'mosapride citrate': 'CHEBI:CHEBI:31867',
  'motor optimality score': 'ungrounded'},
 {'CHEBI:CHEBI:17593': 'maltooligosaccharide',
  'CHEBI:CHEBI:25174': 'mannooligosaccharide',
  'MESH:D018384': 'Oxidative Stress',
  'GO:GO:0036268': 'swimming',
  'CHEBI:CHEBI:31867': 'Mosapride citrate'},
 ['CHEBI:CHEBI:17593',
  'CHEBI:CHEBI:25174',
  'CHEBI:CHEBI:31867',
  'MESH:D018384']]

In [71]:
grounding_map, names, pos_labels = [{'main olfactory system': 'ungrounded',
  'maltooligosaccharide': 'CHEBI:CHEBI:17593',
  'mannan oligosaccharide': 'CHEBI:CHEBI:25174',
  'mannanoligosaccharides': 'CHEBI:CHEBI:25174',
  'manno oligosaccharide': 'CHEBI:CHEBI:25174',
  'mannooligosaccharides': 'CHEBI:CHEBI:25174',
  'margin of stability': 'ungrounded',
  'medical outcome study': 'ungrounded',
  'medical outcome survey': 'ungrounded',
  'metal oxidative semiconductor': 'ungrounded',
  'mitochondrial oxidative stress': 'MESH:D018384',
  'moderate intensity continuous swimming': 'GO:GO:0036268',
  'mosapride citrate': 'CHEBI:CHEBI:31867',
  'motor optimality score': 'ungrounded'},
 {'CHEBI:CHEBI:17593': 'maltooligosaccharide',
  'CHEBI:CHEBI:25174': 'mannooligosaccharide',
  'MESH:D018384': 'Oxidative Stress',
  'GO:GO:0036268': 'swimming',
  'CHEBI:CHEBI:31867': 'Mosapride citrate'},
 ['CHEBI:CHEBI:17593',
  'CHEBI:CHEBI:25174',
  'CHEBI:CHEBI:31867',
  'MESH:D018384']]

In [72]:
excluded_longforms = []

In [73]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [104]:
additional_entities = {}

In [105]:
unambiguous_agent_texts = {}

In [106]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [107]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [108]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [109]:
intersection1

[]

In [110]:
intersection2

[]

In [111]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [112]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [113]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-04 02:55:58] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-04 02:55:59] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.8803409226938639 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [114]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:17593': 4,
  'CHEBI:CHEBI:25174': 63,
  'MESH:D018384': 11,
  'ungrounded': 20,
  'CHEBI:CHEBI:31867': 8,
  'GO:GO:0036268': 1},
 'f1': {'mean': 0.880341, 'std': 0.037938},
 'precision': {'mean': 0.847955, 'std': 0.066131},
 'recall': {'mean': 0.930065, 'std': 0.02389},
 'CHEBI:CHEBI:25174': {'f1': {'mean': 0.920871, 'std': 0.030916},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.854846, 'std': 0.052366}},
 'CHEBI:CHEBI:17593': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0.2, 'std': 0.4},
  'rc': {'mean': 0.2, 'std': 0.4}},
 'ungrounded': {'f1': {'mean': 0.851429, 'std': 0.232397},
  'pr': {'mean': 0.8, 'std': 0.291548},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'CHEBI:CHEBI:31867': {'f1': {'mean': 0.733333, 'std': 0.38873},
  'pr': {'mean': 0.7, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'MESH:D018384': {'f1': {'mean': 0.933333, 'std': 0.133333},
  'pr': {'mean': 0.9, 'std': 0.2},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'GO:GO:0036268

In [115]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [116]:
disamb.dump(model_name, results_path)

In [117]:
print(disamb.info())

Disambiguation model for MOS

Produces the disambiguations:
	MOS*	HGNC:7199
	Mosapride citrate*	CHEBI:CHEBI:31867
	Oxidative Stress*	MESH:D018384
	maltooligosaccharide*	CHEBI:CHEBI:17593
	mannooligosaccharide*	CHEBI:CHEBI:25174
	swimming	GO:GO:0036268

Class level metrics:
--------------------
Grounding           	Count	F1     
mannooligosaccharide*	63	0.92087
          Ungrounded	20	0.85143
    Oxidative Stress*	11	0.93333
   Mosapride citrate*	 8	0.73333
maltooligosaccharide*	 4	    0.2
            swimming	 1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.88034
	Precision:	0.84796
	Recall:		0.93006

* Positive labels
See Docstring for explanation



In [118]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1

In [103]:
corpus[5]

('The objective of this experiment was to compare the effects of dietary MOS and a feed-grade antimicrobial (AM) on growth performance of nursery pigs reared on three different farms (A and B were large-scale commercial farms, and C was located at Michigan State University). On all farms, production was continuous flow by building, but all-in/all-out by room. Within each nursery facility, all pigs on the experiment were in one room. Pigs (Farm A, n = 771, weaning age = 18.4 d; Farm B, n = 576, weaning age = 19.0 d; Farm C, n = 96, weaning age = 20.6 d) were blocked (within farm) by BW and sex and allotted randomly to dietary treatments arranged in a 2 x 2 factorial. The two factors were 1) with and without MOS (0.3% in Phase I, 0.2% in Phases II, III, and IV; as-fed basis) and 2) with and without AM (110 mg of tylosin and 110 mg of sulfamethazine/kg of diet in all phases; as-fed basis). The four nursery phases were 4, 7, 14, and 17 d, respectively. With 35, 20, and 4 pigs per pen on Fa